In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
from shiftreg import ShiftregTester

In [ ]:
ol = ShiftregTester("hw/top_20240824_112221.bit", verbose=True, download=True) # new rx firmware, no ILA, consolidated DMA, interrupt controller

In [ ]:
# check that pgood is fine for all of the pins. if this is nonzero, the AFE isn't working.
0x3f ^ ol.afe_power_status()

# Calibration

In [ ]:
# calibration of dac scale/offset
# get output voltage on 50-ohm load with inputs of -0.125,0,0.125 (quarter of full scale range - doing full scale range introduces nonlinearity which messes up calibration)
# ideally would get a calibration curve to do digital pre-distortion, but that would take too long to manually calibrate, and the analog board doesn't have the necessary components to do an autocal

# step 1:
# set 1/8th of minimum for all outputs, measure voltage
ol.set_dac_scale_offset([0]*8, [-2/8]*8)

In [ ]:
dac_min_voltage_mV = np.array([-12.18, -6.44, -12.85, -7.76] + [-10]*4)

In [ ]:
# step 2:
# set zero for all outputs, measure voltage
ol.set_dac_scale_offset([0]*8, [0]*8)

In [ ]:
dac_zero_voltage_mV = np.array([-3.13, 1.37, -4.20, 0.67] + [0]*4)

In [ ]:
ol.set_dac_scale_offset([0]*8, [2/8]*8)

In [ ]:
dac_max_voltage_mV = np.array([5.78, 9.18, 4.62, 9.07] + [10]*4)

In [ ]:
dac_scale_per_mV = (2/8 - -2/8)/(dac_max_voltage_mV - dac_min_voltage_mV)
dac_bias_correction = -dac_scale_per_mV*(dac_max_voltage_mV + 2*dac_zero_voltage_mV + dac_min_voltage_mV)/4#-dac_zero_voltage_mV*dac_scale_per_mV

In [ ]:
ol.set_dac_scale_offset([0]*8, dac_bias_correction)

In [ ]:
ol.set_dac_scale_offset([0]*8, dac_bias_correction + np.array([3.5, 2, 8, 5] + [0]*4) * dac_scale_per_mV)

In [ ]:
expt_name = 'sr0a_25ohm'
pulse_param_ns = np.array([1, 0, 1, 0, 5])
input_mV = 190e-6*50*1e3
clk1_mV = 190e-6*50*1e3
clk2_mV = 190e-6*50*1e3
clkro_mV = 90e-6*50*1e3
dac_amplitudes_mV = [input_mV, clk1_mV, clk2_mV, clkro_mV]
adc_save_channels = [0, 1, 2, 8]
discriminator_thresholds = [
    [0x8000, 0x8000, 0x8000, 0x7fff], # low
    [0x8000, 0x8000, 0x8000, 0x7fff], # high
    #[0x0, 0x0, 0x0, 0x7fff], # low
    #[0x580, 0x780, 0x200, 0x7fff], # high
]
discriminator_delays = [
    [0, 0, 0, 0], # start (cycles @ 512 MHz)
    [0, 0, 0, 0], # stop
    #[1, 1, 1, 0], # start (cycles @ 512 MHz)
    #[1, 1, 1, 0], # stop
]
discriminator_sources = [0, 1, 2, 3]
adc_atten_dB = [20, 20, 20] + [32]*5
start = time.time()
savefile = ol.single_shiftreg_measurement(expt_name,
                                          adc_save_channels,
                                          pulse_param_ns,
                                          dac_amplitudes_mV,
                                          discriminator_thresholds,
                                          discriminator_delays,
                                          discriminator_sources,
                                          adc_atten_dB,
                                          dac_bias_correction,
                                          dac_scale_per_mV)
stop = time.time()
print(f'runtime = {stop - start} s')

In [ ]:
ol.plot_shiftreg_experiment(savefile, 165e-9, [0.15, 0.8])

In [ ]:
f = np.load(f'data/{savefile}.npz', allow_pickle=True)
active_channels = f['active_channels']
timestamps = f['timestamps']
samples = f['samples']
tvecs = ol._get_tvecs_from_timestamps([0,1,2,3], timestamps, [len(s) for s in samples])

In [ ]:
for tvec in tvecs:
    plt.plot(tvec)

In [ ]:
tstamp_write_depth_packet = [537395712, 3623992576, 8]
tstamp_write_depth_word = sum((d << 32*i) for i,d in enumerate(tstamp_write_depth_packet))
depth_per_bank = []
for i in range(8):
    depth_per_bank.append((tstamp_write_depth_word >> 10*i) & ((1 << 10) - 1))

In [ ]:
depth_per_bank

In [ ]:
[len(s) for s in samples]